
# XGBoost Cleaner (No One-Hot, Categorical Dtypes)

- No use One-Hot；keep **category dtype**，with `XGBRegressor(enable_categorical=True)`。
- Rare category group as `'Other'`；unknown category in prediction also be mapped as `'Other'`。
- Deal with missing value（eg `PoolQC -> NoPool`), Other missing value keep as `NaN`（XGBoost support）。
- feature engineer：`TotalSF`、`AgeSinceBuilt`、`AgeSinceRemod`；cutting area for `GrLivArea`

> recommend using **Parquet** for saving the clean data（reserve dtypes）；CSV is used for preview。


In [8]:

# === Imports & Paths  ===
import os
from typing import List, Tuple, Optional, Dict

import numpy as np
import pandas as pd
from joblib import dump, load
from sklearn.base import BaseEstimator, TransformerMixin

TRAIN_CSV = "../data/train.csv"
TEST_CSV  = "../data/test.csv"
OUTDIR    = "../xgb_clean_outputs"
CLEANER_PATH = f"{OUTDIR}/cleaner_xgb.joblib"

def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def save_df(df: pd.DataFrame, outdir: str, base_name: str, also_csv: bool = True):
    ensure_dir(outdir)
    pq = os.path.join(outdir, base_name if base_name.endswith(".parquet") else f"{base_name}.parquet")
    df.to_parquet(pq, index=False)
    print(f"[OK] Saved Parquet (preserves dtypes): {pq}")
    if also_csv:
        csv = os.path.join(outdir, base_name.replace(".parquet", ".csv") if base_name.endswith(".parquet") else f"{base_name.replace('.csv','')}.csv")
        df.to_csv(csv, index=False)
        print(f"[OK] Saved CSV (for preview only, dtypes lost): {csv}")


## For the columns mapping（Missing-as-Absence）

In [9]:

MISSING_MEANS_NONE = {
    "PoolQC": "NoPool",
    "Alley": "NoAlley",
    "Fence": "NoFence",
    "FireplaceQu": "NoFireplace",
    "GarageType": "NoGarage",
    "GarageFinish": "NoGarage",
    "GarageQual": "NoGarage",
    "GarageCond": "NoGarage",
    "BsmtQual": "NoBasement",
    "BsmtCond": "NoBasement",
    "BsmtExposure": "NoBasement",
    "BsmtFinType1": "NoBasement",
    "BsmtFinType2": "NoBasement",
    "MiscFeature": "None",
    "MasVnrType": "None",
}


## `Rare Category Grouper`

In [10]:

class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    """In each category, if the percentage ot the category less than rare_thresh, group the categories into 'Other' （fit，transform）"""
    def __init__(self, rare_thresh: float = 0.02):
        self.rare_thresh = rare_thresh
        self.frequent_levels_: Dict[str, set] = {}

    def fit(self, X: pd.DataFrame, y=None):
        self.frequent_levels_.clear()
        for col in X.columns:
            vc = X[col].astype("string").fillna(pd.NA).value_counts(normalize=True, dropna=True)
            self.frequent_levels_[col] = set(vc[vc >= self.rare_thresh].index.tolist())
        return self

    def transform(self, X: pd.DataFrame):
        X = X.copy()
        for col in X.columns:
            good = self.frequent_levels_.get(col, set())
            # ONly group for non_missing, missing value is left for XGBoost
            mask = X[col].notna()
            X.loc[mask, col] = X.loc[mask, col].astype("string").where(
                X.loc[mask, col].astype("string").isin(good), other="Other"
            )
        return X


## Special Rules And Features

In [11]:

class SpecialRulesAndFeatures(BaseEstimator, TransformerMixin):
    """
    - MSSubClass → Str（categorial）
    - no garage -> GarageYrBlt=0
    - LotFrontage：filled with median of Neighborhood (else :  NaN）
    - GrLivArea: setting cap
    - feather engineer：TotalSF, AgeSinceBuilt, AgeSinceRemod
    """
    def __init__(self,
                 cap_grlivarea: Optional[float] = 4000.0,
                 create_features: bool = True):
        self.cap = cap_grlivarea
        self.group_median_: Optional[pd.Series] = None
        self.create_features = create_features

    def fit(self, X: pd.DataFrame, y=None):
        if {"LotFrontage", "Neighborhood"}.issubset(X.columns):
            self.group_median_ = X.groupby("Neighborhood")["LotFrontage"].median()
        else:
            self.group_median_ = None
        return self

    def transform(self, X: pd.DataFrame):
        X = X.copy()

        # (1) MSSubClass → str（category）
        if "MSSubClass" in X.columns:
            X["MSSubClass"] = X["MSSubClass"].astype("Int64").astype("string")

        # (2) no garage -> GarageYrBlt = 0
        if "GarageType" in X.columns and "GarageYrBlt" in X.columns:
            no_garage = X["GarageType"].isna() | (X["GarageType"].astype("string").str.lower().isin(["nan", "none"])) 
            X.loc[no_garage, "GarageYrBlt"] = X.loc[no_garage, "GarageYrBlt"].fillna(0)

        # (3) LotFrontage：filled with median of Neighborhood （other: NAN）
        if self.group_median_ is not None and "LotFrontage" in X.columns and "Neighborhood" in X.columns:
            need = X["LotFrontage"].isna()
            X.loc[need, "LotFrontage"] = X.loc[need, "Neighborhood"].map(self.group_median_)

        # (4) GrLivArea : capped
        if self.cap is not None and "GrLivArea" in X.columns:
            X["GrLivArea"] = np.where(
                X["GrLivArea"].notna(), np.minimum(X["GrLivArea"], self.cap), X["GrLivArea"]
            )

        # (5) feature engineer
        if self.create_features:
            for req in ["1stFlrSF", "2ndFlrSF", "TotalBsmtSF"]:
                if req not in X.columns: X[req] = np.nan
            X["TotalSF"] = X["1stFlrSF"] + X["2ndFlrSF"] + X["TotalBsmtSF"]

            for req in ["YrSold", "YearBuilt", "YearRemodAdd"]:
                if req not in X.columns: X[req] = np.nan
            X["AgeSinceBuilt"] = X["YrSold"] - X["YearBuilt"]
            X["AgeSinceRemod"] = X["YrSold"] - X["YearRemodAdd"]

        return X


## XGBCleaner（no use One-Hot，keep `category` with dtype）

In [12]:

class XGBCleaner(BaseEstimator, TransformerMixin):
    def __init__(self,
                 rare_thresh: float = 0.02,
                 cap_grlivarea: Optional[float] = 4000.0,
                 create_features: bool = True,
                 impute_numeric: bool = False):
        self.rare_thresh = rare_thresh
        self.cap = cap_grlivarea
        self.create_features = create_features
        self.impute_numeric = impute_numeric

        self.special_ = SpecialRulesAndFeatures(cap_grlivarea=self.cap, create_features=self.create_features)
        self.rare_ = RareCategoryGrouper(rare_thresh=self.rare_thresh)

        self.cat_cols_: List[str] = []
        self.num_cols_: List[str] = []
        self.cat_levels_: Dict[str, List[str]] = {}

    def _detect_cols(self, X: pd.DataFrame) -> List[str]:
        all_features = [c for c in X.columns if c.lower() != "id"]
        likely_cat = [
            "MSSubClass","MSZoning","Street","LotShape","LandContour","Utilities","LotConfig","LandSlope",
            "Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl",
            "Exterior1st","Exterior2nd","ExterQual","ExterCond","Foundation","Heating","HeatingQC",
            "CentralAir","Electrical","KitchenQual","Functional","PavedDrive","SaleType","SaleCondition"
        ]
        obj_cols = X[all_features].select_dtypes(include=["object"]).columns.tolist()
        cat = list(dict.fromkeys([c for c in likely_cat if c in all_features] + obj_cols))

        # for categorial columns
        cat_none_cols = [c for c in MISSING_MEANS_NONE.keys() if c in all_features]
        cat = list(dict.fromkeys(cat + cat_none_cols))

        # For numerical columns
        num = [c for c in X[all_features].select_dtypes(include=["number"]).columns if c not in cat]

        self.cat_cols_ = cat
        self.num_cols_ = num
        return all_features

    def fit(self, X: pd.DataFrame, y=None):
        Xw = self.special_.fit_transform(X)

        # mapping "missing = none"
        for c, lvl in MISSING_MEANS_NONE.items():
            if c in Xw.columns:
                Xw[c] = Xw[c].astype("string")
                Xw[c] = Xw[c].fillna(lvl).replace({"nan": lvl, "None": lvl})

        self._detect_cols(Xw)

        # rare category group
        cat_df = Xw[self.cat_cols_].copy()
        cat_df = self.rare_.fit_transform(cat_df)

        # record each columns (including 'Other', excluding: missing)
        self.cat_levels_.clear()
        for c in cat_df.columns:
            levels = pd.Series(cat_df[c].dropna().astype("string").unique()).tolist()
            if "Other" not in levels:
                levels.append("Other")
            self.cat_levels_[c] = sorted([str(v) for v in levels])

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        Xw = self.special_.transform(X)

        # mapping “missing = none”
        for c, lvl in MISSING_MEANS_NONE.items():
            if c in Xw.columns:
                Xw[c] = Xw[c].astype("string")
                Xw[c] = Xw[c].fillna(lvl).replace({"nan": lvl, "None": lvl})

        self._detect_cols(Xw)

        # rare category mapping (Prediction: 'Other" when it's not frequent, NaN when missing)
        cat_df = Xw[self.cat_cols_].copy()
        for c in cat_df.columns:
            mask = cat_df[c].notna()
            good = self.rare_.frequent_levels_.get(c, set())
            cat_df.loc[mask, c] = cat_df.loc[mask, c].astype("string").where(
                cat_df.loc[mask, c].astype("string").isin(good), other="Other"
            )

        # transform as category dtype
        for c in cat_df.columns:
            levels = self.cat_levels_.get(c, ["Other"])
            s = cat_df[c].astype("string")
            s = s.where(s.isin(levels), other="Other")
            cat_df[c] = pd.Categorical(s, categories=levels)

        # numerical columns（keep as float）
        num_df = Xw[self.num_cols_].copy()
        if self.impute_numeric:
            for c in num_df.columns:
                if num_df[c].isna().any():
                    num_df[c] = num_df[c].fillna(num_df[c].median())

        # aggregate output
        X_out = pd.concat([num_df, cat_df], axis=1)
        return X_out


## generate cleaner in training data, save as Parquet (keep dytpes)

In [13]:

# === 1) read train.csv ===
ensure_dir(OUTDIR)

df_train = pd.read_csv(TRAIN_CSV)

# target columns
TARGET = "SalePrice"
y = df_train[TARGET].values
X_train_raw = df_train.drop(columns=[TARGET], errors="ignore")

# === 2) 拟合 + 变换 ===
cleaner = XGBCleaner(
    rare_thresh=0.02,
    cap_grlivarea=4000.0,     # no cap if None
    create_features=True,
    impute_numeric=False      # XGBoost can support NaN
)
cleaner.fit(X_train_raw, y)
X_train_clean = cleaner.transform(X_train_raw)

# === 3) Save：Parquet + CSV（preview） ===
save_df(X_train_clean, OUTDIR, "X_clean.parquet", also_csv=True)
pd.DataFrame({"y": y}).to_csv(os.path.join(OUTDIR, "y.csv"), index=False)
with open(os.path.join(OUTDIR, "feature_names.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(map(str, X_train_clean.columns)))
dump(cleaner, CLEANER_PATH)
print(f"[OK] Cleaner saved to: {CLEANER_PATH}")

# Preview
X_train_clean.head()


[OK] Saved Parquet (preserves dtypes): ../xgb_clean_outputs/X_clean.parquet
[OK] Saved CSV (for preview only, dtypes lost): ../xgb_clean_outputs/X_clean.csv
[OK] Cleaner saved to: ../xgb_clean_outputs/cleaner_xgb.joblib


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageFinish,GarageQual,GarageCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,MiscFeature,MasVnrType
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,...,RFn,TA,TA,Gd,TA,No,GLQ,Unf,None,BrkFace
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,...,RFn,TA,TA,Gd,TA,Gd,ALQ,Unf,None,None
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,...,RFn,TA,TA,Gd,TA,Mn,GLQ,Unf,None,BrkFace
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,...,Unf,TA,TA,TA,Gd,No,ALQ,Unf,None,None
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,...,RFn,TA,TA,Gd,TA,Av,GLQ,Unf,None,BrkFace


## Use the same cleaner to preprocess the test.csv（guarantee align with columns/categories）

In [14]:

# === 4) read test.csv，use cleaner to transform ===
df_test = pd.read_csv(TEST_CSV)
cleaner_loaded = load(CLEANER_PATH)
X_submit_clean = cleaner_loaded.transform(df_test)

# save
save_df(X_submit_clean, OUTDIR, "X_submit_clean.parquet", also_csv=True)

# preview
X_submit_clean.head()


[OK] Saved Parquet (preserves dtypes): ../xgb_clean_outputs/X_submit_clean.parquet
[OK] Saved CSV (for preview only, dtypes lost): ../xgb_clean_outputs/X_submit_clean.csv


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageFinish,GarageQual,GarageCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,MiscFeature,MasVnrType
0,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,Unf,TA,TA,TA,TA,No,Rec,LwQ,None,None
1,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,Unf,TA,TA,TA,TA,No,ALQ,Unf,Other,BrkFace
2,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,Fin,TA,TA,Gd,TA,No,GLQ,Unf,None,None
3,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,...,Fin,TA,TA,TA,TA,No,GLQ,Unf,None,BrkFace
4,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,...,RFn,TA,TA,Gd,TA,No,ALQ,Unf,None,None
